In [ ]:
```xml
<!-- filepath: c:\Users\jimbet\Dropbox\Teaching\LLM\LLMsInFinance\src\day4\practical-session\02-simulating-household-decisions.ipynb -->
<VSCode.Cell id="a1b4c3d2" language="markdown">
# Simulating Complex Household Financial Decisions with LLMs

In this notebook, we will explore how to use Large Language Models to simulate complex household financial decision-making processes. We'll demonstrate:
- How to model household financial constraints and preferences
- Techniques for simulating decisions under different economic scenarios
- Methods for analyzing and interpreting LLM-generated financial behavior
- Validation approaches to ensure realistic simulations

## 1. Setup and Dependencies
</VSCode.Cell>
<VSCode.Cell id="e5f6g7h8" language="python">
# Install required packages
!pip install openai pandas numpy matplotlib plotly scikit-learn

# Import libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from openai import OpenAI
import json
import time
from sklearn.cluster import KMeans
from typing import List, Dict, Any
import seaborn as sns
import random
import warnings
warnings.filterwarnings('ignore')

# Initialize OpenAI client (you'll need to set your API key)
# Replace with your preferred LLM API
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "your-api-key-here"))

# Set random seed for reproducibility
np.random.seed(42)
random.seed(42)
</VSCode.Cell>
<VSCode.Cell id="i9j0k1l2" language="markdown">
## 2. Define Household Profiles

We'll create a diverse set of household profiles with varying income levels, assets, debts, family sizes, and financial goals.
</VSCode.Cell>
<VSCode.Cell id="m3n4o5p6" language="python">
# Define a function to create household profiles
def create_household_profiles(num_households=20):
    households = []
    
    # Income distributions (annual)
    income_levels = {
        "low": (25000, 45000),
        "middle": (45001, 120000),
        "high": (120001, 350000),
        "wealthy": (350001, 1000000)
    }
    
    # Asset ranges
    asset_factors = {
        "low": (0.1, 1.0),       # 0.1 to 1x annual income
        "middle": (0.5, 3.0),    # 0.5 to 3x annual income
        "high": (2.0, 10.0),     # 2 to 10x annual income
        "wealthy": (5.0, 30.0)   # 5 to 30x annual income
    }
    
    # Debt ranges
    debt_factors = {
        "low": (0.2, 1.5),       # 0.2 to 1.5x annual income
        "middle": (0.3, 2.0),    # 0.3 to 2x annual income
        "high": (0.2, 1.0),      # 0.2 to 1x annual income
        "wealthy": (0.1, 0.8)    # 0.1 to 0.8x annual income
    }
    
    # Family sizes
    family_sizes = [1, 2, 3, 4, 5, 6]
    family_weights = [0.2, 0.3, 0.2, 0.2, 0.07, 0.03]
    
    # Financial goals
    goals = [
        "Save for retirement",
        "Buy a house",
        "Pay off debt",
        "Save for children's education",
        "Start a business",
        "Build emergency fund",
        "Save for vacation",
        "Invest in stocks/bonds"
    ]
    
    # Risk tolerances
    risk_tolerances = ["Very Conservative", "Conservative", "Moderate", "Aggressive", "Very Aggressive"]
    
    # Generate households
    for i in range(num_households):
        # Select income level
        income_category = random.choices(list(income_levels.keys()), weights=[0.3, 0.4, 0.2, 0.1])[0]
        income_range = income_levels[income_category]
        income = round(random.uniform(*income_range), -3)  # Round to nearest thousand
        
        # Calculate assets and debts
        asset_factor = random.uniform(*asset_factors[income_category])
        debt_factor = random.uniform(*debt_factors[income_category])
        
        assets = round(income * asset_factor, -3)
        debts = round(income * debt_factor, -3)
        
        # Family size
        family_size = random.choices(family_sizes, weights=family_weights)[0]
        
        # Risk tolerance tends to be correlated with income but with variation
        risk_index_base = list(income_levels.keys()).index(income_category)
        risk_adjustment = random.choices([-1, 0, 1], weights=[0.2, 0.6, 0.2])[0]
        risk_index = max(0, min(4, risk_index_base + risk_adjustment + 1))
        risk_tolerance = risk_tolerances[risk_index]
        
        # Primary financial goals - select 1-3 goals
        num_goals = random.randint(1, 3)
        primary_goals = random.sample(goals, num_goals)
        
        # Create household profile
        household = {
            "id": i + 1,
            "income": income,
            "assets": assets,
            "debts": debts,
            "debt_to_income_ratio": round(debts / income, 2),
            "family_size": family_size,
            "risk_tolerance": risk_tolerance,
            "primary_financial_goals": primary_goals,
            "income_category": income_category
        }
        
        households.append(household)
    
    return pd.DataFrame(households)

# Create household profiles
households_df = create_household_profiles(25)
households_df.head()
</VSCode.Cell>
<VSCode.Cell id="q7r8s9t0" language="markdown">
Let's visualize the distribution of our household profiles to ensure we have a diverse set of financial situations.
</VSCode.Cell>
<VSCode.Cell id="u1v2w3x4" language="python">
# Visualize household income distribution
plt.figure(figsize=(12, 6))

# Income distribution
plt.subplot(1, 2, 1)
sns.histplot(households_df['income'], bins=10, kde=True)
plt.title('Household Income Distribution')
plt.xlabel('Annual Income ($)')
plt.ylabel('Count')

# Debt-to-Income ratio by income category
plt.subplot(1, 2, 2)
sns.boxplot(x='income_category', y='debt_to_income_ratio', data=households_df, order=['low', 'middle', 'high', 'wealthy'])
plt.title('Debt-to-Income Ratio by Income Category')
plt.xlabel('Income Category')
plt.ylabel('Debt-to-Income Ratio')

plt.tight_layout()
plt.show()

# Asset vs Debt by family size
plt.figure(figsize=(10, 6))
scatter = plt.scatter(households_df['assets'], households_df['debts'], 
                     c=households_df['family_size'], cmap='viridis', 
                     alpha=0.7, s=100)
plt.colorbar(scatter, label='Family Size')
plt.title('Assets vs Debts by Family Size')
plt.xlabel('Assets ($)')
plt.ylabel('Debts ($)')
plt.grid(True, alpha=0.3)
plt.show()
</VSCode.Cell>
<VSCode.Cell id="y5z6a7b8" language="markdown">
## 3. Define Economic Scenarios

We'll create different economic scenarios that our household simulations will respond to.
</VSCode.Cell>
<VSCode.Cell id="c9d0e1f2" language="python">
# Define economic scenarios
economic_scenarios = {
    "baseline": {
        "name": "Baseline Economy",
        "description": "Current economic conditions with steady growth, 3% inflation, and stable employment.",
        "inflation_rate": 0.03,
        "unemployment_rate": 0.04,
        "interest_rates": {
            "mortgage_30yr": 0.065,
            "auto_loan": 0.07,
            "credit_card": 0.18,
            "savings": 0.035,
            "cd": 0.045
        },
        "stock_market_growth": 0.08,
        "housing_market_growth": 0.05
    },
    "recession": {
        "name": "Economic Recession",
        "description": "Economic downturn with job losses, reduced consumer spending, and market volatility.",
        "inflation_rate": 0.015,
        "unemployment_rate": 0.08,
        "interest_rates": {
            "mortgage_30yr": 0.055,
            "auto_loan": 0.08,
            "credit_card": 0.22,
            "savings": 0.01,
            "cd": 0.02
        },
        "stock_market_growth": -0.15,
        "housing_market_growth": -0.08
    },
    "high_inflation": {
        "name": "High Inflation",
        "description": "Period of elevated inflation with rising prices and central bank intervention.",
        "inflation_rate": 0.075,
        "unemployment_rate": 0.045,
        "interest_rates": {
            "mortgage_30yr": 0.08,
            "auto_loan": 0.095,
            "credit_card": 0.24,
            "savings": 0.045,
            "cd": 0.06
        },
        "stock_market_growth": 0.04,
        "housing_market_growth": 0.02
    },
    "tech_boom": {
        "name": "Technology Boom",
        "description": "Strong economic growth driven by technology sector expansion and innovation.",
        "inflation_rate": 0.025,
        "unemployment_rate": 0.035,
        "interest_rates": {
            "mortgage_30yr": 0.07,
            "auto_loan": 0.065,
            "credit_card": 0.175,
            "savings": 0.04,
            "cd": 0.05
        },
        "stock_market_growth": 0.18,
        "housing_market_growth": 0.09
    }
}

# Convert to DataFrame for easier viewing
scenarios_df = pd.DataFrame(economic_scenarios).T
scenarios_df = scenarios_df.drop('interest_rates', axis=1)
scenarios_df
</VSCode.Cell>
<VSCode.Cell id="g3h4i5j6" language="markdown">
## 4. LLM-Based Household Decision Simulation

Now we'll create a simulation using an LLM to model how different households would make financial decisions under various economic scenarios.
</VSCode.Cell>
<VSCode.Cell id="k7l8m9n0" language="python">
def simulate_household_decisions(household, scenario, model="gpt-4-turbo"):
    """
    Use an LLM to simulate financial decisions made by a household under a given economic scenario
    
    Args:
        household: Dictionary with household information
        scenario: Dictionary with economic scenario information
        model: LLM model to use
    
    Returns:
        Dictionary with simulated decisions and reasoning
    """
    # Format the prompt with household and scenario information
    prompt = f"""You are a financial planning expert tasked with simulating realistic financial decisions for a household.
    
HOUSEHOLD PROFILE:
- Annual Income: ${household['income']:,.0f}
- Assets: ${household['assets']:,.0f}
- Debts: ${household['debts']:,.0f}
- Debt-to-Income Ratio: {household['debt_to_income_ratio']}
- Family Size: {household['family_size']}
- Risk Tolerance: {household['risk_tolerance']}
- Primary Financial Goals: {', '.join(household['primary_financial_goals'])}

ECONOMIC SCENARIO - {scenario['name']}:
{scenario['description']}
- Inflation Rate: {scenario['inflation_rate']:.1%}
- Unemployment Rate: {scenario['unemployment_rate']:.1%}
- Mortgage Rate (30yr): {scenario['interest_rates']['mortgage_30yr']:.2%}
- Savings Account Rate: {scenario['interest_rates']['savings']:.2%}
- Stock Market Expected Growth: {scenario['stock_market_growth']:.1%}
- Housing Market Expected Growth: {scenario['housing_market_growth']:.1%}

Based on the household profile and economic scenario, simulate this household's financial decisions over the next year.

Output your response as a JSON object with the following structure:
{{
    "monthly_budget": {{
        "housing": dollar_amount,
        "transportation": dollar_amount,
        "food": dollar_amount,
        "utilities": dollar_amount,
        "healthcare": dollar_amount,
        "debt_payments": dollar_amount,
        "savings": dollar_amount,
        "investments": dollar_amount,
        "education": dollar_amount,
        "entertainment": dollar_amount,
        "other": dollar_amount
    }},
    "financial_decisions": [
        {{
            "decision": "Brief description of decision",
            "reasoning": "Explanation for why this decision makes sense for this household"
        }},
        ...
    ],
    "investment_allocation": {{
        "stocks": percentage,
        "bonds": percentage,
        "cash": percentage,
        "real_estate": percentage,
        "other": percentage
    }},
    "risk_management": [
        "Strategy 1 to manage risk in this scenario",
        "Strategy 2 to manage risk in this scenario",
        ...
    ],
    "expected_financial_outcome": "Overall expected outcome for this household in this scenario"
}}

Ensure the monthly budget items add up to the household's monthly income, and investment percentages sum to 100%.
"""

    # Call the LLM API
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are a financial planning expert who understands household economics."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.2,  # Low temperature for more deterministic outputs
            response_format={"type": "json_object"}  # Request JSON output
        )
        
        # Parse the JSON response
        decision_data = json.loads(response.choices[0].message.content)
        
        # Add metadata
        decision_data["household_id"] = household["id"]
        decision_data["income_category"] = household["income_category"]
        decision_data["scenario"] = scenario["name"]
        
        return decision_data
    
    except Exception as e:
        print(f"Error simulating household {household['id']}: {e}")
        return None
</VSCode.Cell>
<VSCode.Cell id="o1p2q3r4" language="markdown">
Let's run the simulation for a subset of households across different economic scenarios.
</VSCode.Cell>
<VSCode.Cell id="s5t6u7v8" language="python">
# Select a subset of households for simulation (one from each income category)
sample_households = []
for category in ['low', 'middle', 'high', 'wealthy']:
    sample_households.append(
        households_df[households_df['income_category'] == category].iloc[0].to_dict()
    )

# Run simulations
simulation_results = []

for household in sample_households:
    for scenario_name, scenario in economic_scenarios.items():
        print(f"Simulating household {household['id']} in {scenario_name} scenario...")
        
        # Sleep to avoid rate limiting
        time.sleep(1)
        
        # Run simulation
        result = simulate_household_decisions(household, scenario)
        if result:
            simulation_results.append(result)

# Convert results to DataFrame
simulation_df = pd.DataFrame(simulation_results)
</VSCode.Cell>
<VSCode.Cell id="w9x0y1z2" language="markdown">
## 5. Analyze Simulation Results

Now we'll analyze and visualize the simulation results to understand how different households respond to economic scenarios.
</VSCode.Cell>
<VSCode.Cell id="a3b4c5d6" language="python">
# Extract budget data for analysis
budget_data = []

for result in simulation_results:
    household_id = result['household_id']
    scenario = result['scenario']
    income_category = result['income_category']
    
    for category, amount in result['monthly_budget'].items():
        budget_data.append({
            'household_id': household_id,
            'scenario': scenario,
            'income_category': income_category,
            'budget_category': category,
            'amount': amount
        })

budget_df = pd.DataFrame(budget_data)

# Analyze how budget allocations change across scenarios
plt.figure(figsize=(15, 10))

# Group by income category and scenario, get mean budget allocation
pivot_df = budget_df.pivot_table(
    index=['income_category', 'scenario'], 
    columns='budget_category', 
    values='amount',
    aggfunc='mean'
)

# Normalize to percentages
normalized_pivot = pivot_df.div(pivot_df.sum(axis=1), axis=0) * 100

# Plot heatmap
plt.figure(figsize=(16, 10))
sns.heatmap(normalized_pivot, annot=True, fmt='.1f', cmap='YlGnBu', linewidths=0.5)
plt.title('Budget Allocation (%) by Income Category and Economic Scenario')
plt.ylabel('Income Category - Scenario')
plt.xticks(rotation=45, ha='right')
plt.show()

# Compare investment allocations across scenarios
investment_data = []

for result in simulation_results:
    household_id = result['household_id']
    scenario = result['scenario']
    income_category = result['income_category']
    
    for asset_class, percentage in result['investment_allocation'].items():
        investment_data.append({
            'household_id': household_id,
            'scenario': scenario,
            'income_category': income_category,
            'asset_class': asset_class,
            'percentage': percentage
        })

investment_df = pd.DataFrame(investment_data)

# Plot investment allocations
plt.figure(figsize=(14, 8))
sns.barplot(x='scenario', y='percentage', hue='asset_class', data=investment_df)
plt.title('Investment Allocation by Economic Scenario')
plt.xlabel('Economic Scenario')
plt.ylabel('Allocation Percentage')
plt.xticks(rotation=45)
plt.legend(title='Asset Class')
plt.show()
</VSCode.Cell>
<VSCode.Cell id="e7f8g9h0" language="markdown">
## 6. Validate Simulation Results

Let's validate the quality of our LLM-generated simulations by comparing them with some basic financial heuristics.
</VSCode.Cell>
<VSCode.Cell id="i1j2k3l4" language="python">
def validate_simulation_results(simulation_results, households_df):
    """
    Validate the simulation results against basic financial heuristics
    """
    validation_results = []
    
    for result in simulation_results:
        household_id = result['household_id']
        scenario = result['scenario']
        
        # Get the corresponding household data
        household = households_df[households_df['id'] == household_id].iloc[0]
        
        # Calculate monthly income
        monthly_income = household['income'] / 12
        
        # Validation checks
        budget_sum = sum(result['monthly_budget'].values())
        investment_sum = sum(result['investment_allocation'].values())
        
        # Check if budget exceeds income
        budget_income_ratio = budget_sum / monthly_income
        
        # Check if savings aligns with risk tolerance
        savings_rate = result['monthly_budget']['savings'] / monthly_income
        
        # Check if debt payments are reasonable
        min_debt_payment = household['debts'] * 0.03 / 12  # Minimum 3% monthly payment on debt
        debt_payment = result['monthly_budget']['debt_payments']
        
        # Calculate validation scores (0-1, higher is better)
        budget_validation = 1.0 if 0.95 <= budget_income_ratio <= 1.05 else max(0, 1 - abs(budget_income_ratio - 1))
        investment_validation = 1.0 if 99 <= investment_sum <= 101 else max(0, 1 - abs(investment_sum - 100) / 100)
        debt_validation = 1.0 if debt_payment >= min_debt_payment else debt_payment / min_debt_payment
        
        # Get risk tolerance score (1-5)
        risk_map = {
            "Very Conservative": 1,
            "Conservative": 2,
            "Moderate": 3, 
            "Aggressive": 4,
            "Very Aggressive": 5
        }
        risk_score = risk_map.get(household['risk_tolerance'], 3)
        
        # Assess if investment allocation matches risk tolerance
        stocks_pct = result['investment_allocation'].get('stocks', 0)
        conservative_assets = result['investment_allocation'].get('bonds', 0) + result['investment_allocation'].get('cash', 0)
        
        # Higher risk score should correlate with higher stock allocation
        expected_stocks_min = (risk_score - 1) * 15  # E.g., risk score 5 -> min 60% stocks
        expected_stocks_max = expected_stocks_min + 20  # E.g., risk score 5 -> max 80% stocks
        
        if expected_stocks_min <= stocks_pct <= expected_stocks_max:
            risk_validation = 1.0
        else:
            risk_validation = max(0, 1 - min(abs(stocks_pct - expected_stocks_min), abs(stocks_pct - expected_stocks_max)) / 100)
        
        # Calculate overall validation score
        overall_score = (budget_validation * 0.3 + 
                         investment_validation * 0.2 + 
                         debt_validation * 0.3 + 
                         risk_validation * 0.2)
        
        validation_results.append({
            'household_id': household_id,
            'income_category': household['income_category'],
            'scenario': scenario,
            'budget_validation': budget_validation,
            'investment_validation': investment_validation,
            'debt_validation': debt_validation,
            'risk_validation': risk_validation,
            'overall_validation_score': overall_score
        })
    
    return pd.DataFrame(validation_results)

# Validate our simulation results
validation_df = validate_simulation_results(simulation_results, households_df)

# Visualize validation results
plt.figure(figsize=(12, 8))
validation_pivot = validation_df.pivot_table(
    index='income_category', 
    columns='scenario', 
    values='overall_validation_score',
    aggfunc='mean'
)

sns.heatmap(validation_pivot, annot=True, fmt='.2f', cmap='RdYlGn', vmin=0, vmax=1, linewidths=0.5)
plt.title('Simulation Validation Scores by Income Category and Economic Scenario')
plt.xlabel('Economic Scenario')
plt.ylabel('Income Category')
plt.show()

# Show detailed validation metrics
validation_metrics = validation_df[['income_category', 'scenario', 'budget_validation', 
                                   'investment_validation', 'debt_validation', 
                                   'risk_validation', 'overall_validation_score']]
validation_metrics
</VSCode.Cell>
<VSCode.Cell id="m5n6o7p8" language="markdown">
## 7. Extract Financial Insights

Let's analyze the LLM-generated financial decisions to extract insights about household behavior across economic scenarios.
</VSCode.Cell>
<VSCode.Cell id="q9r0s1t2" language="python">
# Extract financial decisions for analysis
decision_data = []

for result in simulation_results:
    household_id = result['household_id']
    scenario = result['scenario']
    income_category = result['income_category']
    
    for decision in result['financial_decisions']:
        decision_data.append({
            'household_id': household_id,
            'scenario': scenario,
            'income_category': income_category,
            'decision': decision['decision'],
            'reasoning': decision['reasoning']
        })

decision_df = pd.DataFrame(decision_data)

# Display sample of decisions for each scenario
for scenario in economic_scenarios.keys():
    print(f"\n--- Sample Decisions in {scenario.upper()} Scenario ---")
    sample = decision_df[decision_df['scenario'] == economic_scenarios[scenario]['name']].sample(min(3, len(decision_df)))
    for _, row in sample.iterrows():
        print(f"[{row['income_category'].upper()} Income Household] {row['decision']}")
        print(f"   Reasoning: {row['reasoning']}\n")

# Create a wordcloud of common terms in decisions
from wordcloud import WordCloud

plt.figure(figsize=(12, 8))

# Combine all decisions into a single text
all_decisions = " ".join(decision_df['decision'].tolist())

# Generate and display wordcloud
wordcloud = WordCloud(width=800, height=400, background_color='white', 
                     max_words=100, contour_width=3, contour_color='steelblue')
wordcloud.generate(all_decisions)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Common Terms in Financial Decisions')
plt.show()
</VSCode.Cell>
<VSCode.Cell id="u3v4w5x6" language="markdown">
## 8. Building a Financial Decision Simulator Function

Let's encapsulate our simulation into a reusable function that can be used to generate financial decision insights for any household profile under any economic scenario.
</VSCode.Cell>
<VSCode.Cell id="y7z8a9b0" language="python">
def financial_decision_simulator(household_profile, economic_scenario, model="gpt-4-turbo"):
    """
    A financial decision simulator that predicts how a household would behave under a given economic scenario.
    
    Args:
        household_profile: Dictionary with household information including income, assets, debts, etc.
        economic_scenario: Dictionary with economic scenario information
        model: LLM model to use
    
    Returns:
        Dictionary with simulated financial decisions and detailed budget
    """
    # Validate inputs
    required_household_fields = ['income', 'assets', 'debts', 'family_size', 'risk_tolerance']
    required_scenario_fields = ['name', 'description', 'inflation_rate', 'unemployment_rate', 'interest_rates']
    
    for field in required_household_fields:
        if field not in household_profile:
            raise ValueError(f"Missing required household field: {field}")
    
    for field in required_scenario_fields:
        if field not in economic_scenario:
            raise ValueError(f"Missing required scenario field: {field}")
    
    # Calculate debt-to-income ratio if not provided
    if 'debt_to_income_ratio' not in household_profile:
        household_profile['debt_to_income_ratio'] = round(household_profile['debts'] / household_profile['income'], 2)
    
    # Run the simulation
    result = simulate_household_decisions(household_profile, economic_scenario, model)
    
    if not result:
        return {"error": "Simulation failed"}
    
    # Validate the simulation results
    monthly_income = household_profile['income'] / 12
    budget_sum = sum(result['monthly_budget'].values())
    budget_diff_pct = abs((budget_sum - monthly_income) / monthly_income) * 100
    
    investment_sum = sum(result['investment_allocation'].values())
    investment_diff = abs(investment_sum - 100)
    
    # Add validation information
    result['validation'] = {
        "monthly_income": monthly_income,
        "budget_sum": budget_sum,
        "budget_difference_percent": budget_diff_pct,
        "investment_allocation_sum": investment_sum,
        "investment_allocation_difference": investment_diff,
        "is_budget_balanced": budget_diff_pct <= 5,  # Allow 5% margin of error
        "is_investment_balanced": investment_diff <= 2  # Allow 2% margin of error
    }
    
    return result

# Example usage
custom_household = {
    "id": 100,
    "income": 85000,
    "assets": 120000,
    "debts": 65000,
    "family_size": 3,
    "risk_tolerance": "Moderate",
    "primary_financial_goals": ["Save for retirement", "Pay off debt"],
    "income_category": "middle"
}

simulation = financial_decision_simulator(custom_household, economic_scenarios["high_inflation"])

# Display the simulation results
print(f"Financial Decision Simulation for a {custom_household['income_category'].capitalize()} Income Household")
print(f"Economic Scenario: {simulation['scenario']}\n")

print("Monthly Budget:")
for category, amount in simulation['monthly_budget'].items():
    print(f"  {category.capitalize()}: ${amount:,.2f}")

print("\nInvestment Allocation:")
for asset, percentage in simulation['investment_allocation'].items():
    print(f"  {asset.capitalize()}: {percentage}%")

print("\nKey Financial Decisions:")
for i, decision in enumerate(simulation['financial_decisions'], 1):
    print(f"  {i}. {decision['decision']}")

print("\nExpected Outcome:")
print(f"  {simulation['expected_financial_outcome']}")

print("\nValidation:")
print(f"  Budget Balanced: {simulation['validation']['is_budget_balanced']}")
print(f"  Investment Allocation Balanced: {simulation['validation']['is_investment_balanced']}")
</VSCode.Cell>
<VSCode.Cell id="c1d2e3f4" language="markdown">
## 9. Conclusion

In this notebook, we've demonstrated how to use Large Language Models to simulate complex household financial decision-making processes. We've shown how to:

1. Create diverse household financial profiles
2. Define realistic economic scenarios
3. Use LLMs to simulate financial decisions
4. Analyze and validate simulation results
5. Extract insights from LLM-generated financial behavior
6. Build a reusable financial decision simulator

This approach can be valuable for:
- Financial planning and advisory services
- Economic policy impact assessment
- Consumer behavior research
- Financial education and planning tools

By leveraging LLMs, we can create realistic simulations of financial decision-making that take into account a wide range of factors, including household characteristics, economic conditions, and financial goals.
</VSCode.Cell>
</VSCode>```